Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = "Adam Bakopolus"
COLLABORATORS = "N/A"

# Presenting Uncertainty
## School of Information, University of Michigan

## Week 2: Assignment Overview
Version 1.1
### The objectives for this week are for you to:

- review the concept of standard error
- construct a confidence distribution
- use a confidence distribution to construct a density plot, interval plot, CCDF barplot, and quantile dotplot
- apply one or more of the above techniques to the World Happiness dataset

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import altair as alt
import random
from scipy import stats
from scipy.stats import norm

# Part 1. Standard error and confidence distributions (14 points)

## 1.1 Bootstrap sampling distribution of phone pickups example

Recall in assignment 1 we used a sample from a hypothetical dataset of counts of how many times students at a school picked up their phones. Let's generate such a hypothetical dataset again. We'll make a sample of 20 students and the number of times each student picked up their phone:

In [3]:
np.random.seed(1234)   # for reproducibility

pickups_df = pd.DataFrame({"pickups": np.random.poisson(lam=40, size=20)})

pickups_df.describe()

,pickups
count,20.000000
mean,39.350000
std,6.072154
min,27.000000
25%,36.750000
50%,38.500000
75%,43.250000
max,52.000000


### Question 1.1.1 (2 points)

Using what you learned in assignment 1, generate a bootstrap sampling distribution of the mean number of pickups and assign it to the variable `bootstrap_means`:

In [4]:
B = 5000
np.random.seed(1234) # for reproducibility

bootstrap_means = []
# construct the bootstrap sampling distribution of the mean of you sample (`sample_pickups`)
# and store it in `bootstrap_means`
# Hint: use np.random.choice()

for b in range(B):
    bootstrap_mean = np.mean(np.random.choice(pickups_df['pickups'], 20))
    bootstrap_means.append(bootstrap_mean)
#raise NotImplementedError()

In [5]:
# the bootstrap sampling mean should be close to the sample mean
sample_mean = pickups_df['pickups'].mean()
sample_mean_se = np.std(pickups_df['pickups'], ddof=1) / np.sqrt(len(pickups_df))
assert len(bootstrap_means) == 5000, "Bootstrap sampling distribution: length of bootstrap_means should be 5000"
assert np.abs(np.std(bootstrap_means) - sample_mean_se) / sample_mean_se < 0.05, "Bootstrap sampling distribution: SD of boostrap_means does not match SE of sample_pickups"
assert np.abs(np.mean(bootstrap_means) - sample_mean)/sample_mean < 5*1e-3, "Bootstrap sampling distribution: mean of bootstrap_means does not match sample_mean"

### Question 1.1.2 (2 points)

Visualize the bootstrap sampling distribution of the mean as a density plot.

Hint: see assignment 1.

In [6]:
#Bootstrap Density Plot

boot = pd.DataFrame(data = {"boot_pickups" : bootstrap_means})

boot_dens_chart = alt.Chart(boot).transform_density(
    density='boot_pickups',
    as_=['boot_pickups', 'density'],
    
).mark_area(color="lightgray").encode(
    x=alt.X('boot_pickups', axis=alt.Axis(title="Bootstrap sampling distribution of mean pickups")),
    y=alt.Y('density:Q', axis=alt.Axis(title="density"))
).properties(width = 500, height = 100)

boot_m_chart = alt.Chart(boot).mark_rule(color="red", strokeDash=[1,1]).encode(
    x="mean(boot_pickups)"
)

pickups_bootstrap_chart = boot_dens_chart + boot_m_chart

pickups_bootstrap_chart
#raise NotImplementedError()

alt.LayerChart(...)

## 1.2 Standard error and Normal sampling distribution for mean pickups

The sampling distribution of a mean approaches the shape of a Normal distribution as the sample size increases. Instead of using bootstrapping to derive the distribution, we could instead use the *standard error*, which is an estimate of the standard deviation of the sampling distribution, assuming the sampling distribution is Normal.

The standard error of the mean ($s_\bar{x}$) is calculated using the sample standard deviation ($s$) and the sample size ($n$):

$s_\bar{x} = \frac{s}{\sqrt{n}}$

This calculation can be done as follows:

In [7]:
# manual calculation of standard error
pickups_se = pickups_df['pickups'].std(ddof=1) / np.sqrt(len(pickups_df))
print("Manual calculation of SE:           ", pickups_se)

# or using scipy.stats.sem() (should be the same)
pickups_se = stats.sem(pickups_df['pickups'])
print("scipy.stats.sem() calculation of SE:", pickups_se)


Manual calculation of SE:            1.357774882511437
scipy.stats.sem() calculation of SE: 1.357774882511437


Given the sample mean pickups ($\bar{x}$ = `pickups_mean`) and the sample mean standard error ($s_\bar{x}$ = `pickups_se`), we can construct a sampling distribution using the standard error. As discussed in lecture, there are three functions we'll want to define in order to construct different uncertainty visualizations: the probability density function (PDF), the cumulative distribution function (CDF), and the quantile function (also called the percentage points function, PPF). For a mean and standard error, these are:

$\begin{align}
\mathrm{density~function~(PDF):~} & f_\mathrm{Normal}\left(x \middle| \bar{x}, s_\bar{x}\right) & \mathtt{stats.norm.pdf()}\\
\mathrm{cumulative~distribution~function~(CDF):~} & F_\mathrm{Normal}\left(x \middle| \bar{x}, s_\bar{x}\right) & \mathtt{stats.norm.cdf()}\\
\mathrm{quantile~function~(PPF):~} & F^{-1}_\mathrm{Normal}\left(p \middle| \bar{x}, s_\bar{x}\right) & \mathtt{stats.norm.ppf()}
\end{align}$

In [8]:
pickups_mean = pickups_df['pickups'].mean()

# generate evenly-spaced numbers covering the range of values of pickups
# that we will calculate the density of the sampling distribution at
x = np.linspace(
    start = pickups_df['pickups'].min(), 
    stop = pickups_df['pickups'].max(),
    num = 1001
)

# this is the density of the sampling distribution: f_Normal(x | x_bar, se)
density = stats.norm.pdf(x, pickups_mean, pickups_se)

df = pd.DataFrame({"Normal sampling distribution of mean pickups": x, "density": density})

pickups_se_chart = alt.Chart(df).mark_line().encode(
    x="Normal sampling distribution of mean pickups",
    y="density"
)

pickups_se_chart

alt.Chart(...)

## 1.3 t-based confidence distribution for mean pickups

A slight improvement to the sampling distribution above is to use a Student's t confidence distribution. At small sample sizes, the Normal approximation is less accurate; a scaled-and-shifted Student's t distribution will have slightly fatter tails than the Normal distribution. 

To determine how fat the tails are, the t distribution uses a *degrees of freedom* parameter, which for the estimate of a single independent mean of sample size $n$ is equal to $n - 1$. Thus, an improved confidence distribution over the Normal approximation above is:

$\begin{align}
\mathrm{density~function~(PDF):~} & f_\mathrm{t}\left(x \middle| n - 1, \bar{x}, s_\bar{x}\right) & \mathtt{stats.t.pdf()}\\
\mathrm{cumulative~distribution~function~(CDF):~} & F_\mathrm{t}\left(x \middle| n - 1, \bar{x}, s_\bar{x}\right) & \mathtt{stats.t.cdf()}\\
\mathrm{quantile~function~(PPF):~} & F^{-1}_\mathrm{t}\left(p \middle| n - 1, \bar{x}, s_\bar{x}\right) & \mathtt{stats.t.ppf()}
\end{align}$

This confidence distribution is related to the common Student's t test: confidence intervals from this distribution are the same as the confidence intervals you would generate from a t test.

### Question 1.3.1 Visualize the Student's t confidence distribution for the mean (5 points)

Visualize the density of the t-based confidence distribution described above in a similar manner to how the Normal sampling distribution was visualized, but use a different color for the line (this will help you with the next question):


In [9]:
density2 = stats.t.pdf(x, df=len(pickups_df) - 1, loc=pickups_mean, scale=pickups_se)

df2 = pd.DataFrame({"T-based sampling distribution of mean pickups": x, "density": density2})

pickups_t_chart = alt.Chart(df2).mark_line(color = 'red').encode(
    x="T-based sampling distribution of mean pickups",
    y="density"
)

pickups_t_chart
#raise NotImplementedError()

alt.Chart(...)

### Question 1.3.2 Visualize and compare uncertainty distributions (5 points)

Visualize all three distributions you have constructed so far in a single plot. 

Hint: if you have saved your previous plots in three separate variables and made them with different enough encodings that they can be distinguished by eye, this answer can be as simple as `pickups_bootstrap_chart + pickups_se_chart + pickups_t_chart`

In [10]:
pickups_bootstrap_chart+pickups_se_chart+pickups_t_chart
#raise NotImplementedError()

alt.LayerChart(...)

Compare these distributions: what do you notice? Write your answer below.

While these 3 plots, when placed together, do look to be very similar, there are a couple of differences worth noting. First, for the bootstrap plot, unlike the normal and t-confidence plots, the bootstrap plot is the least "sharp" peak-wise around the expected mean. There is much larger density over a range of points around this expected value when compared to the other plots, which would be expected when a random sample with replacement from an already small sample is used to generate a density plot.

For the normal and t-confidence plots, these look very similar at first glance, as well, but the one difference to note is that the normal plot has a sharper peak and, similarly, flattens out slightly quicker than the t-confidence plot. This would be expected as the degrees of freedom configuration tied to t-confidence plots leads to fatter tails so as to account for smaller samples that may not truly reflect (yet) a normal distribution.

# Part 2. Intervals, CDFs, and quantile dotplots (17 points)

Now that we are able to define the density, CDF, and quantile functions to describe our uncertainty in a mean, we can use these to construct various uncertainty visualizations as described in the lectures. For example, we can use the density function to create density plots or gradients; the CDF to create complementary CDF bar plots; and the quantile function to create intervals and quantile dotplots.

## 2.1 Intervals

### Question 2.1.1 Calculate 95% interval (2 points)

Using the quantile function (aka percentage points function) of the Student's t confidence distribution you derived in Part 1, calculate the lower and upper bounds of a 95% confidence interval for mean pickups and assign these bounds to the `pickups_lower` and `pickups_upper` variables. 

(Hint: use `stats.t.ppf()` as the quantile function and see the lecture on intervals.)

In [11]:
pickups_lower = stats.t.ppf(0.025, df=len(pickups_df) - 1, loc=pickups_mean, scale=pickups_se)
pickups_upper = stats.t.ppf(0.975, df=len(pickups_df) - 1, loc=pickups_mean, scale=pickups_se)
#raise NotImplementedError()

In [12]:
dof = len(pickups_df) - 1
m = pickups_df['pickups'].mean()
se = stats.sem(pickups_df['pickups'])
#hiddent tests below

### Question 2.1.2 Visualize 95% + 66.666% interval (5 points)

Visualize a combined 95% + 66.666% (i.e. 2/3rds) interval for the mean of pickups, overlaid on top of the data (as ticks). 

Hints: 
- create two different layers, one for the 95% interval and one for the 66.666% interval, then combine them using `alt.layer()` or `+`.
- use `mark_rule(size=XXX).encode(x=YYY, x2=ZZZ)` to create each interval, where you fill in the values of `size` (to set the thickness of the interval), `x` (to set the lower bound), and `x2` (to set the upper bound).

Your output should look like this:

![Plot of 66% and 95% confidence intervals overliad on data](asset/assignment2_intervals.png)

In [13]:
sample_vals = alt.Chart(pickups_df).mark_rule(color="red").encode(
    alt.X("pickups", axis=alt.Axis(title="lower")))

pickups_lower66 = stats.t.ppf(0.167, df=len(pickups_df) - 1, loc=pickups_mean, scale=pickups_se)
pickups_upper66 = stats.t.ppf(0.833, df=len(pickups_df) - 1, loc=pickups_mean, scale=pickups_se)

ranges = [[pickups_lower, pickups_upper, pickups_lower66, pickups_upper66]]
ranges_df = pd.DataFrame(ranges, columns = ['lower95', 'upper95', 'lower66', 'upper66'])

chart95 = alt.Chart(ranges_df).mark_rule(size = 3).encode(x = 'lower95', x2 = 'upper95')
chart66 = alt.Chart(ranges_df).mark_rule(size = 7).encode(x = 'lower66', x2 = 'upper66')

sample_vals+chart95+chart66
   
#raise NotImplementedError()

alt.LayerChart(...)

## 2.2 Cumulative distribution function

### Question 2.2.1 Visualize a Complementary CDF barplot (5 points)

Using the CDF function (or the CCDF function), plot a CCDF barplot for the Student's t-based confidence distribution. *The barplot should start at 0*.

Hint: start from the code for generating a density plot, above, and (1) adjust the code for generating `x` to ensure it includes 0 and (2) change the function from a PDF to 1 - the CDF.

Your output should look something like this:

![CCDF barplot of mean pickups](asset/assignment2_ccdf.png)

In [14]:
x_cdf = np.linspace(
    start = 0, 
    stop = pickups_df['pickups'].max(),
    num = 1001
)

density_cdf = stats.t.cdf(x_cdf, df=len(pickups_df) - 1, loc=pickups_mean, scale=pickups_se)

df_cdf = pd.DataFrame({"Mean pickups (CCDF)": x_cdf, "ccdf": 1-density_cdf})

pickups_cdf_chart = alt.Chart(df_cdf).mark_area().encode(
    x="Mean pickups (CCDF)",
    y="ccdf"
).properties(width = 500, height = 50)

pickups_cdf_chart
#raise NotImplementedError()

alt.Chart(...)

## 2.3 Quantile dotplots

Constructing quantile dotplots involves first generating a small-to-medium number of quantiles from the quantile function (aka percentage points function) of the uncertainty distribution. We do this by projecting evenly-spaced values in probability space back into the data units through the quantile function, then stacking up those values into a dotplot:

<img src="asset/cdf.png" alt="wo graphs stacked on top of each other. The top graph has a y-axis of p less than x, from 0.00 to 1.00 in intervals of 0.25, and the x-axis is x, from 0 to 30. The cumulative distribution function line starts out around (0, 0.00) and stays that way until about (8, 0.00) where it rises up the y axis until it reaches about (18, 1.00) and continues on to 30. The bottom graph has a y-axis of count, from 0.00 to 1.00 in intervals of 0.25, and the x-axis is x, from 0 to 30. Circles are stacked between 8 and 18 on the x-axis, with a peak around (11, 0.50) and is slightly skewed to the right." style="width: 500px;"/>

The first step is to generate the evenly-spaced probabilities. We will create a `ppoints(n)` function that generates `n` evenly-spaced probabilities (not including 0 and 1):

In [15]:
def ppoints(n):
    return (np.arange(n) + 0.5)/n

ppoints(20)

array([0.025, 0.075, 0.125, 0.175, 0.225, 0.275, 0.325, 0.375, 0.425,
       0.475, 0.525, 0.575, 0.625, 0.675, 0.725, 0.775, 0.825, 0.875,
       0.925, 0.975])

We can then use `ppoints` with the quantile function of the t confidence distribution (`stats.t.ppf`) to construct a quantile dotplot.

Dotplots in Altair are complex to construct, requiring three transformations:

1. `transform_bin` to bin the values
2. `transform_window` to rank the values within each bin (this stacks up all the dots in one bin)
3. `transform_joinaggregate` to calculate the midpoint of each bin using the median of all values in that bin

Putting together `ppoints`, the quantile function of the confidence distribution, and the above steps, we can construct a quantile dotplot.

In [16]:
df = pd.DataFrame({
    "x": stats.t.ppf(ppoints(50), len(pickups_df) - 1, pickups_mean, pickups_se)
})

alt.Chart(df).transform_bin(
    field="x",
    as_="bin",
    bin=alt.BinParams(step=0.75)
).transform_window(
    rank_in_bin="rank()",
    groupby=["bin"]
).transform_joinaggregate(
    bin_midpoint="median(x)",
    groupby=["bin"]
).mark_circle(size=90).encode(
    x="bin_midpoint:Q",
    y="rank_in_bin:Q"
).properties(width=600, height=100)

alt.Chart(...)

### Question 2.3.1 Construct a 20-dot quantile dotplot for the mean of pickups (5 points)

Construct a 20-dot quantile dotplot for the mean of pickups. Adjust the binning step size, circle size, and/or chart size to achieve a closely-packed quantile dotplot like the 50-dot dotplot above. It is typically also necessary to adjust the step size in the binning (`alt.BinParams(step=XXX)`), the size of the circles (`mark_circle(size=XXX)`) and/or the width and height of the plot (`properties(width=XXX, height=YYY)`) to achieve good-looking dotplots.

In [17]:
df_ppf = pd.DataFrame({
    "x": stats.t.ppf(ppoints(20), len(pickups_df) - 1, pickups_mean, pickups_se)
})

alt.Chart(df_ppf).transform_bin(
    field="x",
    as_="bin",
    bin=alt.BinParams(step=1.5) #.75
).transform_window(
    rank_in_bin="rank()",
    groupby=["bin"]
).transform_joinaggregate(
    bin_midpoint="median(x)",
    groupby=["bin"]
).mark_circle(size=120).encode( #90
    x="bin_midpoint:Q",
    y="rank_in_bin:Q"
).properties(width=400, height=100) #600, 100
#raise NotImplementedError()

alt.Chart(...)

## Part 3. Visualize world happiness (10 points)

We will do some visualization of the World Happiness Report dataset from 2015. It is a survey of the state of global happiness, which ranks 155 countries by their happiness levels. Let's explore the happiness score.

First, we'll read in the data and gather the happiness score for all countries:

In [18]:
#Read in 2015 World Happiness Report statistics as a dataframe
happ = pd.read_csv("asset/2015.csv")

#show the top 15 countries:
happ.head(15)

,Country,Region,Happiness Rank,Happiness Score,Standard Error,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
0,Switzerland,Western Europe,1,7.587,0.03411,1.39651,1.34951,0.94143,0.66557,0.41978,0.29678,2.51738
1,Iceland,Western Europe,2,7.561,0.04884,1.30232,1.40223,0.94784,0.62877,0.14145,0.43630,2.70201
2,Denmark,Western Europe,3,7.527,0.03328,1.32548,1.36058,0.87464,0.64938,0.48357,0.34139,2.49204
3,Norway,Western Europe,4,7.522,0.03880,1.45900,1.33095,0.88521,0.66973,0.36503,0.34699,2.46531
4,Canada,North America,5,7.427,0.03553,1.32629,1.32261,0.90563,0.63297,0.32957,0.45811,2.45176
5,Finland,Western Europe,6,7.406,0.03140,1.29025,1.31826,0.88911,0.64169,0.41372,0.23351,2.61955
6,Netherlands,Western Europe,7,7.378,0.02799,1.32944,1.28017,0.89284,0.61576,0.31814,0.47610,2.46570
7,Sweden,Western Europe,8,7.364,0.03157,1.33171,1.28907,0.91087,0.65980,0.43844,0.36262,2.37119
8,New Zealand,Australia and New Zealand,9,7.286,0.03371,1.25018,1.31967,0.90837,0.63938,0.42922,0.47501,2.26425
9,Australia,Australia and New Zealand,10,7.284,0.04083,1.33358,1.30923,0.93156,0.65124,0.35637,0.43562,2.26646


In [19]:
#some descriptive statistics of the World Happiness Report:
happ.describe()

,Happiness Rank,Happiness Score,Standard Error,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
count,158.000000,158.000000,158.000000,158.000000,158.000000,158.000000,158.000000,158.000000,158.000000,158.000000
mean,79.493671,5.375734,0.047885,0.846137,0.991046,0.630259,0.428615,0.143422,0.237296,2.098977
std,45.754363,1.145010,0.017146,0.403121,0.272369,0.247078,0.150693,0.120034,0.126685,0.553550
min,1.000000,2.839000,0.018480,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.328580
25%,40.250000,4.526000,0.037268,0.545808,0.856823,0.439185,0.328330,0.061675,0.150553,1.759410
50%,79.500000,5.232500,0.043940,0.910245,1.029510,0.696705,0.435515,0.107220,0.216130,2.095415
75%,118.750000,6.243750,0.052300,1.158448,1.214405,0.811013,0.549092,0.180255,0.309883,2.462415
max,158.000000,7.587000,0.136930,1.690420,1.402230,1.025250,0.669730,0.551910,0.795880,3.602140


## 3.1 Happiness Scores

We'll focus on two columns in particular: the `Happiness Score` (an aggregate happiness score for each country) and the `Standard Error`, which is the standard error of the `Happiness Score`: because the data are based on surveys, there is uncertainty in the measurement of the happiness score, and this is quantified using its standard error.

### Question 3.1.1 Visualize the happiness scores and uncertainty of the top 15 countries (5 points)

Use one of the uncertainty visualizations above to visualize the happiness score of each country. You will need to use the Normal approximation to the sampling distribution along with the standard error of the happiness score to do this.

You **MUST NOT** use just a combination of a point and a single interval to visualize the scores. For example, the following chart **IS NOT AN ACCEPTABLE SOLUTION** (but could be a good place to start):

![point estimates and 95% confidence intervals for the happiness score in the top 10 countries according to the World Happiness Report](asset/assignment2_happiness_intervals.png)

But you could construct a chart with a similar layout, but which uses one of the uncertainty encodings you learned above. For example, you might start by constructing a chart with 95% intervals, then extend it to be a chart with 95%+66.666% intervals, which would be an acceptable solution.

Hints:

1. You may want to sort the y axis by happiness score using something like `alt.Y("Country", sort=["Happiness Score"]`
2. You may want to restrict the x axis domain to make it easier to see the uncertainty using something like `alt.X("Happiness Score", scale={"domain":[6,8]})`
3. If you decide to make density plots, quantile dotplots, or CCDF barplots, you need to use the `facet()` function to display multiple subcharts where each one is a single country. If you do that, you may want to set the `"labelAngle"` property on the facet headers so they are horizontal.

In [20]:
happ_top = happ.head(15)

happ_top["Happiness"] = happ_top['Happiness Score']
happ_top["Error"] = happ_top['Standard Error']

happ_top['lower66'] = happ_top.apply(lambda row: stats.norm.ppf(0.167, loc=row.Happiness, scale=row.Error), axis = 1)
happ_top['upper66'] = happ_top.apply(lambda row: stats.norm.ppf(0.833, loc=row.Happiness, scale=row.Error), axis = 1)

happ_top['lower95'] = happ_top.apply(lambda row: stats.norm.ppf(0.025, loc=row.Happiness, scale=row.Error), axis = 1)
happ_top['upper95'] = happ_top.apply(lambda row: stats.norm.ppf(0.975, loc=row.Happiness, scale=row.Error), axis = 1)

happ_circle = alt.Chart(happ_top).mark_circle(color="red", size = 50).encode(
    alt.X("Happiness Score", axis=alt.Axis(title="Happiness Score"), scale={"domain":[6,8]}),
    alt.Y("Country", sort=["Happiness Score"])
).properties(title = 'Happiness Score of Top 15 Countries with 95% and 66% Standard Error Intervals')

happ95 = alt.Chart(happ_top).mark_rule(size = 3).encode(
    x = alt.X('lower95', scale={"domain":[6,8]}), 
    x2 = 'upper95', 
    y = alt.Y('Country', sort=['Happiness Score']))

happ66 = alt.Chart(happ_top).mark_rule(size = 7).encode(
    x = alt.X('lower66', scale={"domain":[6,8]}), 
    x2 = 'upper66', 
    y = alt.Y('Country', sort=['Happiness Score']))

happ95+happ66+happ_circle
#raise NotImplementedError()

alt.LayerChart(...)

### Question 3.1.2 Reflect on the chart you created (5 points)

What are the pros and cons of the uncertainty encoding you used? How might it compare to other options you could have chosen?

The above plot has the top 15 countries encoded on the y-axis and sorted based on their Happiness Score from various surveys from high to low. The Happiness Score itself is encoded by x-position and color (red) in order to help distinguish it from the uncertainty intervals. Based on the standard error tied to the various surveys used to generate the Happiness Score, 66% and 95% uncertainty intervals are encoded by position on the x-axis. The 66% interval is also encoded a bit thicker than the 95% interval to help differentiate the two for the reader.

As discussed in the lectures, while certain plots like dot plots or CDFs, when applicable, are perhaps easier to perceive by a reader to understand uncertainty/probability, the above plot is still a very effective means of presenting the uncertainty findings in addition to the Happiness Score itself. First, position is easy to perceive by viewers and the spatial relationship between countries and their Happiness Score is expressed effectively with this encoding. The intervals are also effective in displaying the 66% and 95% uncertainty more so than if opacity, for example, was encoded to display this information. The reader would be able to estimate these uncertainty ranges using the x-axis very effectively, and it has been discussed in lectures that this ability is significantly diminished when opacity is used. As a result, this uncertainty encoding, while perhaps not the ideal choice when compared to CDFs or dot plots, is still a better selection over opacity and succeeds in its ability to effectively display the uncertainty tied to the Happiness Score.

Please remember to submit both the HTML and .ipynb formats of your completed notebook. When generating your HTML, be sure to run your complete code first before downloading as HTML. Please remember to work on your explanations and interpretations!